<h1><center>Bioptim Workshop</center></h1>

# Diving further into *Bioptim*: a receding horizon application

Let's play with *Bioptim* in a motion tracking context, with the intention to use it in real time.
Who said it was not possibe to solve large, constrained, non-linear optimization problems in real time ?!
*Acados* can do such a thing, and *Bioptim* is your friend to help you formulate such a challenging problem in biomechanics.

The outline of this example is as follows:

* First, we will generate simulated marker positions data for an arm model, as if they had been recorded by a motion capture system.
* Then, we will sequentially solve small optimization problems to retrieve muscle control and joint kinematics in pseudo real time.

Our example should work as if each time a new dataframe is acquired, the dynamics and the kinematics of the model is optimized accordingly.
Such a formulation is called *moving horizon estimation*, and it is pre-implemented in bioptim !


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import casadi as cas
import biorbd_casadi as biorbd
from scipy.integrate import solve_ivp

from bioptim import *

In [ ]:
## First this function allows to symbolically compute the markers positions from the joints positions
def states_to_markers(model, states):
    nq = model.nb_q
    nmark = model.nb_markers
    q = cas.MX.sym("q", nq)
    markers_func = biorbd.to_casadi_func("markers", model.markers, q)
    markers = np.array(markers_func(states[:nq, :]))
    return markers.reshape((3, nmark, -1), order="F")  # reshape to XYZ x n_mark x n_instants


In [ ]:
## Let's generate simulated data with our arm model
model = BiorbdModel("models/arm26.bioMod")
nq = model.nb_q
ndq = model.nb_qdot
ntau = model.nb_tau
nmark = model.nb_markers
tf = 1
n_shoot_per_sec = 100
n_shoot = n_shoot_per_sec*tf
t_max = 2
x0 = np.zeros((nq + ndq,))

# Create a symbolic version of the forward dynamics of the model
q = cas.MX.sym("q", nq)
qdot = cas.MX.sym("qdot", nq)
tau = cas.MX.sym("tau", nq)
qddot_func = biorbd.to_casadi_func("forw_dyn", model.forward_dynamics, q, qdot, tau)

def arm_ode(t, x, u):
    return np.concatenate((x[nq:, np.newaxis], qddot_func(x[:nq], x[nq:], u)))[:, 0]


# Simulated data
dt = tf / n_shoot
controls = np.zeros((ntau, n_shoot))  # Control trajectory
controls[0, :] = (-np.ones(n_shoot) + np.sin(np.linspace(0, tf, num=n_shoot))) * t_max
states = np.zeros((nq + ndq, n_shoot))  # State trajectory

for n in range(n_shoot):
    sol = solve_ivp(arm_ode, [0, dt], x0, args=(controls[:, n],))
    states[:, n] = x0
    x0 = sol["y"][:, -1]
states[:, -1] = x0
markers = states_to_markers(model, states[:nq, :])

# Simulated noise
np.random.seed(42)
noise_std = 0.01 # 1cm of noise std on each marker
noise = (np.random.randn(3, nmark, n_shoot) - 0.5) * noise_std
markers_noised = markers + noise

fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
q_plot = plt.plot(states[:nq, :].T)
dq_plot = plt.plot(states[nq:, :].T, "--")
plt.legend(q_plot + dq_plot, list(model.name_dof) + ["d" + name for name in model.name_dof])
plt.title("Real position and velocity trajectories of the simulated arm")

fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
marker_plot = plt.plot(markers_noised[2, :, :].T, markers_noised[1, :, :].T, "x")
plt.legend(marker_plot, model.marker_names)
plt.gca().set_prop_cycle(None)
plt.plot(markers[2, :, :].T, markers[1, :, :].T, "b")
plt.title("2D plot of markers trajectories + noise")
plt.axis('equal')
plt.show()

Now that we have created this simulated dataset, we can optimize !

In [ ]:
model = BiorbdModel("models/arm26.bioMod")
window_len = 10
window_duration = 1 / n_shoot_per_sec * window_len
n_frames_total = tf * n_shoot_per_sec - window_len - 1
x_init = np.zeros((nq * 2, window_len + 1))
u_init = np.zeros((ntau, window_len))
torque_max = 8  # Give a bit of slack on the torque bound

new_objectives = Objective(ObjectiveFcn.Lagrange.MINIMIZE_MARKERS, weight=1000, list_index=0)
mhe = MovingHorizonEstimator(
        model,
        Dynamics(DynamicsFcn.TORQUE_DRIVEN),
        window_len,
        window_duration,
        objective_functions=new_objectives,
        x_init=InitialGuess(x_init, interpolation=InterpolationType.EACH_FRAME),
        u_init=InitialGuess(u_init, interpolation=InterpolationType.EACH_FRAME),
        x_bounds=model.bounds_from_ranges(["q", "qdot"]),
        u_bounds=Bounds([-torque_max]*ntau, [torque_max]*ntau),
        n_threads=4,
    )

def update_functions(mhe, t, _):
    def target(i: int):
        return markers_noised[:, :, i : i + window_len + 1]

    mhe.update_objectives_target(target=target(t), list_index=0)
    return t < n_frames_total  # True if there are still some frames to reconstruct

# Solve the program
solver = Solver.ACADOS()
solver.set_maximum_iterations(20)
solver.set_integrator_type("ERK")
solver.set_print_level(1)

sol = mhe.solve(update_functions, solver)



In [ ]:
print("ACADOS with Bioptim")
print(f"Window size of MHE : {window_duration} s.")
print(f"New measurement every : {1/n_shoot_per_sec} s.")
print(f"Average time per iteration of MHE : {sol.solver_time_to_optimize/(n_frames_total - 1)} s.")
print(f"Average real time per iteration of MHE : {sol.real_time_to_optimize/(n_frames_total - 1)} s.")
print(f"Norm of the error on state = {np.linalg.norm(states[:,:n_frames_total] - sol.states['all'])}")

markers_estimated = states_to_markers(model, sol.states["all"])

fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.plot(
    markers_noised[1, :, :n_frames_total].T,
    markers_noised[2, :, :n_frames_total].T,
    "x",
    label="Noised markers trajectory",
)
plt.plot(markers_estimated[1, :, :].T, markers_estimated[2, :, :].T, "bo", label="Estimated marker trajectory")
plt.plot(markers[1, :, :n_frames_total].T, markers[2, :, :n_frames_total].T, 'k', label="True markers trajectory")
plt.legend()

fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.plot(sol.states["q"].T, "x", label="Joint positions estimate")
plt.gca().set_prop_cycle(None)
plt.plot(states[:nq, :n_frames_total].T, 'k', label="Joint positions truth")
plt.legend()
plt.show()

fig=plt.figure(figsize=(12,8), dpi= 100, facecolor='w', edgecolor='k')
plt.plot(sol.states["qdot"].T, "x", label="Joint velocities estimate")
plt.gca().set_prop_cycle(None)
plt.plot(states[nq:, :n_frames_total].T, 'k',label="Joint velocities truth")
plt.legend()
plt.show()